In [1]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from groq import Groq
import openai

In [2]:
import minsearch

## Ingestion

In [3]:
df = pd.read_csv('../data/data.csv')

In [4]:
documents = df.to_dict(orient='records')

In [5]:
index = minsearch.Index(
    text_fields=['exercise_name', 'type_of_activity', 'type_of_equipment', 'body_part',
       'type', 'muscle_groups_activated', 'instructions'],
    keyword_fields=['id']
)

In [6]:
index.fit(documents)

## Rag Flow

In [7]:
load_dotenv()

False

In [8]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [9]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [10]:
prompt_template = """
You're a fitness instructor. Answer the QUESTION based on the CONTEXT from our exercises database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

entry_template = """ 
exercise_name : {exercise_name}
type_of_activity : {type_of_activity}
type_of_equipment : {type_of_equipment}
body_part : {body_part}
type : {type}
muscle_groups_activated : {muscle_groups_activated}
instructions : {instructions}
""".strip()
    
def build_prompt(query, search_results):
    
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [44]:
search_results = search(query)
prompt = build_prompt(query, search_results)

In [49]:
def llm(prompt):
    response = client.chat.completions.create(
        model="gemma2-9b-it",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [56]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt)
    return answer

In [57]:
query =  "Can you explain how to do a Glute Bridge, I'm not sure about the movement."

In [58]:
answer = rag(query)
print(answer)

To do a Glute Bridge, lie on your back with your feet flat on the ground. Lift your hips by squeezing your glutes until you form a straight line from your shoulders to your knees. Then, slowly lower your hips back down to the starting position.  

